# Even reduction

In [1]:
using DynamicPolynomials
@polyvar x

(x,)

We would like to find the minimum value of the following polynomial:

In [2]:
poly = x^4 - 2x^2

using SumOfSquares

We define the custom action as follows:

In [3]:
struct OnSign <: Symmetry.OnMonomials end
using PermutationGroups
import SymbolicWedderburn
SymbolicWedderburn.coeff_type(::OnSign) = Float64
function SymbolicWedderburn.action(::OnSign, p::Permutation, mono::AbstractMonomial)
    if isone(p) || iseven(DynamicPolynomials.degree(mono))
        return 1 * mono
    else
        @assert p.perm == perm"(1,2)"
        return -1 * mono
    end
end
G = PermGroup([perm"(1,2)"])

Permutation group on 1 generator generated by
 (1,2)

We can exploit the symmetry as follows:

In [4]:
import CSDP
solver = CSDP.Optimizer
model = Model(solver)
@variable(model, t)
@objective(model, Max, t)
pattern = Symmetry.Pattern(G, OnSign())
con_ref = @constraint(model, poly - t in SOSCone(), symmetry = pattern)
optimize!(model)
value(t)

Stuck at edge of primal feasibility, giving up. 
Partial Success: SDP solved with reduced accuracy
Primal objective value: -9.3223355e-01 
Dual objective value: -9.3303494e-01 
Relative primal infeasibility: 3.50e-08 
Relative dual infeasibility: 2.36e-10 
Real Relative Gap: -2.80e-04 
XZ Relative Gap: 1.18e-09 
DIMACS error measures: 4.59e-08 0.00e+00 6.23e-10 0.00e+00 -2.80e-04 1.18e-09
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj: -7.7333326e+00 Ad: 9.00e-01 Dobj:  5.2199992e+00 
Iter:  2 Ap: 1.00e+00 Pobj: -4.7469915e+00 Ad: 9.00e-01 Dobj:  4.9208659e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -1.3207096e+00 Ad: 9.00e-01 Dobj: -8.2374194e-01 
Iter:  4 Ap: 1.00e+00 Pobj: -1.0090262e+00 Ad: 9.00e-01 Dobj: -9.8744356e-01 
Iter:  5 Ap: 9.00e-01 Pobj: -1.0008968e+00 Ad: 1.00e+00 Dobj: -9.9998051e-01 
Iter:  6 Ap: 1.00e+00 Pobj: -1.0001857e+00 Ad: 1.00e+00 Dobj: -9.9990815e-01 
Iter:  7 Ap: 9.00e-01 Pobj: -1.0000185e+00 Ad: 1.00

-1.0000000070845787

We indeed find `-1`, let's verify that symmetry was exploited:

In [5]:
gram_matrix(con_ref)

BlockDiagonalGramMatrix with 2 blocks:
[1] Block with row/column basis:
     FixedPolynomialBasis([1.0, x^2])
    And entries in a 2×2 SymMatrix{Float64}:
      1.0000000070845412  -1.0000000020057762
     -1.0000000020057762   1.0000000000001108
[2] Block with row/column basis:
     FixedPolynomialBasis([x])
    And entries in a 1×1 SymMatrix{Float64}:
     4.011410370015888e-9

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*